### Closure Applications (Part 1)

#### 此Notebook為Aaron的注釋版本

We are going to build average func that can average multiple values

The twist is that we want to simply be able to feed numbers to that function and get a running average over time. Not average a list which requires performing same calculations (sum and count) over and over again

In [ ]:
def Averager(): ## Aaron注意錯誤，有沒有括弧沒關係，錯誤是def，是函式，class才是定義物件。
    def __init__(self):
        self.numbers = []
        
    def add(self, number):
        self.numbers.append(number)
        total = sum(self.numbers)
        count = len(self.numbers)
        return total / count

In [18]:
a = Averager()

In [19]:
a.add(10)

AttributeError: 'NoneType' object has no attribute 'add'

In [3]:
class Averager:
    def __init__(self):
        self.numbers = []
    
    def add(self, number):
        self.numbers.append(number)
        total = sum(self.numbers)
        count = len(self.numbers)
        return total / count

In [4]:
a = Averager()

In [5]:
a.add(10)

10.0

In [6]:
a.add(20)

15.0

In [7]:
a.add(30)

20.0

In [9]:
# b is a differnt object

b = Averager()
b.add(10)

10.0

#### 利用函數（不是object）試試看：

In [11]:
def averager():
    numbers = []
    def add(number):
        numbers.append(number)
        total = sum(numbers)
        count = len(numbers)
        return total / count
    
    # 這是錯誤的，因為忘記/沒有 return closure （return add）
        

In [12]:
fn = averager()

In [13]:
fn

In [14]:
fn()

TypeError: 'NoneType' object is not callable

In [16]:
def averager():
    numbers = []
    def add(number):
        numbers.append(number)
        total = sum(numbers)
        count = len(numbers)
        return total / count
    return add  # this is returning closure

In [17]:
a = averager()

In [18]:
a(10)

10.0

In [19]:
a(20)

15.0

In [20]:
a(30)

20.0

In [21]:
b = averager()

In [22]:
b(10)

10.0

In [23]:
a.__closure__

(<cell at 0x10663bac0: list object at 0x1074b97c0>,)

In [24]:
b.__closure__

(<cell at 0x106972440: list object at 0x1067a77c0>,)

#### rewrite - so that we won't calc all sum and average each time. The new sum is the previous sum add the new number

In [25]:
def averager():
    total = 0
    count = 0
    def add(number):
        nonlocal total
        nonlocal count
        total = total + number
        count = count + 1
        return total / count

    return add

In [26]:
a = averager()

In [27]:
a.__closure__

(<cell at 0x106795030: int object at 0x1010f6f90>,
 <cell at 0x106797010: int object at 0x1010f6f90>)

In [28]:
a.__code__.co_freevars

('count', 'total')

In [29]:
a(10)

10.0

In [30]:
a(20)

15.0

In [31]:
a(30)

20.0

#### 現在implement object

In [ ]:
class Averager():
    def __init__(self):
        self.total = 0
        self.count = 0

    def add(self, number):
        total = self.total + number  # aaron: this is wrong!! should be self.total
        count = self.count + 1  # aaron: this is wrong, should be self.count
        return self.total / self.count
        

In [39]:
a = Averager()

In [40]:
a

In [41]:
a.add(10)

ZeroDivisionError: division by zero

In [36]:
a.add(10)

ZeroDivisionError: division by zero

In [44]:
class Averager():
    def __init__(self):
        self.total = 0
        self.count = 0

    def add(self, number):
        self.total = self.total + number
        self.count = self.count + 1
        return  self.total / self.count

In [45]:
a = Averager()
a.add(10)

10.0

In [46]:
a.add(20)

15.0

In [47]:
a.add(20)

16.666666666666668

#### counter

In [48]:
from time import perf_counter

In [49]:
perf_counter()

415717.480814416

In [50]:
perf_counter()

415724.235016583

In [51]:
class Timer:
    def __init__(self):
        self.start = perf_counter()

    def poll(self):
        return perf_counter() - self.start

In [52]:
t1 = Timer()

In [53]:
t1.poll()

6.853146250010468

In [54]:
t1.poll()

13.060682291979901

#### 修改上面的Timer，的add，為callable

In [56]:
class Timer():
    def __init__(self):
        self.start = perf_counter()

    def __call__(self):
        return perf_counter() - self.start

In [57]:
t1 = Timer()

In [58]:
t1

In [59]:
t1()

5.879983040969819

In [60]:
t1()

10.201796207984444

#### The equivalent function to class

In [61]:
def timer():
    start = perf_counter()
    def poll():
        return perf_counter() - start
    return poll

In [62]:
t2 = timer()

In [63]:
t2()

5.31195437500719

In [64]:
t2()

8.26946283399593

In [25]:
class Averager1():
    def __init__(self):
        self._count = 0
        self._total = 0
    
    def add(self, value):
        self._total += value
        self._count += 1
        return self._total / self._count

class Averager2:
    def __init__(self):
        self._count = 0
        self._total = 0
    
    def add(self, value):
        self._total += value
        self._count += 1
        return self._total / self._count

a1 = Averager1()
a2 = Averager2()

print(a1.add(10))  # 10.0
print(a2.add(10))  # 10.0

10.0
10.0


#### 下面兩個class的不同之處

> -  這兩個 Averager 類別的行為相似，但內部的計算方式不同，因此可能會影響性能與數據存儲方式。
> - 不過，對於 a.add(10), a.add(20), a.add(30) 來說，它們的輸出結果是一樣的。

##### 第一種
🔹 特點

> - self._total 只存總和，self._count 只存數量。
> - 空間效率高：只存兩個變數 _total 和 _count。
> - 計算效率高：每次 add() 只做一次加法和除法，O(1) 常數時間 計算。


In [37]:
class Averager:
    def __init__(self):
        self._count = 0
        self._total = 0
    
    def add(self, value):
        self._total += value  # 直接累加總和
        self._count += 1  # 計數 +1
        return self._total / self._count  # 計算平均值

In [36]:
a = Averager()
print(a.add(10))
print(a.add(20))
print(a.add(30))

10.0
15.0
20.0


##### 第二種


🔹 特點

> - self.numbers 存儲所有輸入過的數字。
> - 空間效率較差：每個新數值都要存下來，資料量大時記憶體使用量高。
> - 計算效率較低：每次 add() 會重新計算 sum(self.numbers) 和 len(self.numbers)，O(n) 線性時間 計算。

In [37]:
class Averager:
    def __init__(self):
        self.numbers = []  # 存儲所有的數值
    
    def add(self, number):
        self.numbers.append(number)  # 把新數值加入列表
        total = sum(self.numbers)  # 計算總和
        count = len(self.numbers)  # 計算數量
        return total / count  # 計算平均值

In [38]:
a = Averager()
print(a.add(10))
print(a.add(20))
print(a.add(30))

10.0
15.0
20.0


In [39]:
class Averager():
    def __init__(self):
        self._total = 0
        self._count = 0

    def add(self, value):
        self._total += value
        self._count += 1
        return self._total / self._count

In [40]:
class Averager2():
    def __init__(self):
        self.numbers = []

    def add(self, number):
        self.numbers.append(number)
        total = sum(self.numbers)
        count = len(self.numbers)
        return total / count

In [41]:
a = Averager()
a.add(10)
a.add(20)
a.add(30)

20.0

In [42]:
a2 = Averager2()
a2.add(10)
a2.add(20)
a2.add(30)

20.0

### Aaron - Class 與 Closure 的比較
這兩種 `class` 的寫法都可以改寫成使用 **closure（閉包）/自由變數** 的函數來實現。這裡我們會 **逐步轉換** 並比較哪一種方式更好！

---

#### **📌 方法 1：用 Closure 改寫第一種 class**
##### **原本的 class**
這種方式使用 `_total` 和 `_count` 來累積計算：


In [43]:
class Averager:
    def __init__(self):
        self._count = 0
        self._total = 0
    
    def add(self, value):
        self._total += value
        self._count += 1
        return self._total / self._count


#### **改寫為 Closure**


In [44]:
def make_averager():
    _count = 0
    _total = 0

    def add(value):
        nonlocal _count, _total  # 使用 nonlocal 讓內部函數修改外部變數
        _total += value
        _count += 1
        return _total / _count
    
    return add  # 返回 add 函數作為 API

# 使用方式
a = make_averager()
print(a(10))  # 10.0
print(a(20))  # 15.0
print(a(30))  # 20.0


10.0
15.0
20.0



🔹 **重點解析**
- `_count` 和 `_total` 被封裝在 `make_averager()` 的作用域中。
- `add()` 是一個閉包，透過 `nonlocal` 來更新 `_count` 和 `_total`。
- `make_averager()` **返回 `add()` 函數**，這樣 `a = make_averager()` 就像創建了一個物件。



### **📌 方法 2：用 Closure 改寫第二種 class**
#### **原本的 class**
這種方式使用 `self.numbers` 儲存所有數據：


In [45]:
class Averager:
    def __init__(self):
        self.numbers = []
    
    def add(self, number):
        self.numbers.append(number)
        total = sum(self.numbers)
        count = len(self.numbers)
        return total / count



#### **改寫為 Closure**


In [46]:
def make_averager():
    numbers = []

    def add(number):
        numbers.append(number)  # 記錄所有輸入值
        total = sum(numbers)  # 計算總和
        count = len(numbers)  # 計算數量
        return total / count
    
    return add

# 使用方式
a = make_averager()
print(a(10))  # 10.0
print(a(20))  # 15.0
print(a(30))  # 20.0


10.0
15.0
20.0



🔹 **重點解析**
- `numbers` 變數存儲所有的輸入值。
- `add()` 是一個閉包，能夠訪問 `numbers`，並計算平均值。

---

### **📊 哪一種方式比較好？**
| 方式 | 儲存方式 | 計算效率 | 記憶體使用 | 適用場景 |
|------|---------|---------|---------|---------|
| **方式 1** (`_total`, `_count`) | **只記錄總和 & 次數** | **O(1) (快速)** | **O(1) (少量)** | 適合大數據量、高效計算 |
| **方式 2** (`numbers`) | **存所有數據** | **O(n) (慢，因為每次都要 sum())** | **O(n) (較多)** | 適合需要保留歷史數據的場景 |

#### **🚀 總結**
1. **如果你不需要保存所有數據，只想計算平均數，方式 1 (`_total` 和 `_count`) 更好**，因為它的**計算速度快** (`O(1)`) 且不占用太多記憶體 (`O(1)`)。
2. **如果你需要記錄每個輸入值（比如要查詢歷史記錄），方式 2 (`numbers` 列表) 會更適合**，但它的 `sum()` 會讓計算變慢 (`O(n)`)。

✅ **建議**：
- **大數據場景** → 用 `_total` & `_count`，避免 `sum()` 造成計算瓶頸。
- **需要保留完整數據** → 用 `numbers` 列表，但記憶體消耗較高。

---

### **📌 `class` 和 `closure` 哪種方式更好？**
| 方式 | 易讀性 | 可擴展性 | 多功能性 | 典型使用情境 |
|------|--------|--------|--------|--------|
| **Class (OOP)** | **較好** | **較好** | **較好** | 更適合大型專案，方便擴展 |
| **Closure (Functional)** | **簡潔** | **較差** | **單一功能** | 適合小型函數封裝，輕量級應用 |

#### **🚀 結論**
1. 如果你的程式很簡單，**只是一個小工具**，可以考慮用 **closure**，因為它簡潔又方便。
2. 如果你可能**需要擴展功能（例如添加 reset()、統計中位數等）**，用 **class** 會更靈活。

**💡 實務上，大部分專案會用 `class`，因為它更易讀、更容易擴展**！但 `closure` 是一種有趣的 Python 技巧，適合小型功能封裝。 🚀

> Aaron's Experiments on `Closure Application - Part1`

In this example, we are going to build an `averager` function that can average multiple values.

The twist is that we want to simply be able to feed numbers to that function and get a running average over time, not average a list which requires performinng the same calculations (sum and count) over and over again.

In [1]:
class Averager:
    def __init__(self):
        self.numbers = []

    def add(self, number):
        self.numbers.append(number)
        total = sum(self.numbers)
        count = len(self.numbers)
        return total / count

In [2]:
a = Averager()

In [4]:
a.add(10)

10.0

In [5]:
a.add(20)

15.0

In [6]:
a.add(30)

20.0

We can do this using a closure as follows:

In [7]:
def averager():
    numbers = []
    def add(number):
        numbers.append(number)
        total = sum(numbers)
        count = len(numbers)
        return total / count
    return add

In [8]:
a = averager()

In [10]:
a.__closure__, a.__code__.co_freevars

((<cell at 0x1057abe50: list object at 0x1057cbc80>,), ('numbers',))

In [11]:
a(10)

10.0

In [12]:
a(20)

15.0

In [13]:
a(30)

20.0

Now, instead of storing a list and recalculating `total` and `count` every time, we need the new average.

We are going to store the ruuning total and count and update each value each time a new value is added to the running average, and then return `total / count`

Let's start with a class approach first.

We will use instance variables to store the running total and count. And provide an instance method to add a new number and return the current average.

In [23]:
class Averager:
    def __init__(self):
        self._count = 0
        self._total = 0
    
    def add(self, value):
        self._total += value
        self._count += 1
        return self._total / self.count # this var has typo _count

In [18]:
a = Averager()

In [19]:
a

In [20]:
a.add()

TypeError: Averager.add() missing 1 required positional argument: 'value'

In [24]:
class Averager:
    def __init__(self):
        self._count = 0
        self._total = 0
    
    def add(self, value):
        self._total += value
        self._count += 1
        return self._total / self._count

In [25]:
a = Averager()


In [26]:
a.add(10)

10.0

In [27]:
a.add(20)

15.0

In [28]:
a.add(30)

20.0

Now, let's see how we might use a `closure` to achieve the same thing.

In [29]:
def average():
    total = 0
    count = 0

    def add(value):
        nonlocal total, count
        total += value
        count += 1
        return 0 if count == 0 else total / count
    
    return add

In [30]:
a = averager()

In [31]:
a(10)

10.0

In [32]:
a(20)

15.0

In [33]:
a(30)

20.0

### `Generalizing this example`

We saw that we were essentially able to convert to a class to an equivalent functionality using closures.

This is acutally true in a much more general sense - very often, `classes` that define a single method (other than initializers) can be implemented using a closure instead.

Let's look at another example of this

Suppoose we want something that can keep track of the running elapsed time in seconds.

In [34]:
from time import perf_counter

In [35]:
class Timer:
    def __init__(self):
        self._start = perf_counter()

    def __call__(self):
        return (perf_counter() - self._start)

In [36]:
a = Timer()

Now wait a bit before running the next line of code:

In [37]:
a()

20.029656084007

In [38]:
class Timer:
    def __init__(self):
        self._start = perf_counter()
    
    def __call__(self):
        return (perf_counter() - self._start)

a = Timer()

In [39]:
a()

6.290960083002574

In [40]:
a()

13.011587833010708

Let's start another `timer`

In [41]:
b = Timer()

In [42]:
print(a())
print(b())

54.35045195800194
12.455509292005445


Now let's rewrite this using a closure instead:

In [43]:
def timer():
    start = perf_counter()

    def elapsed():
        # we dont even need to make start nonlocal
        # since we are only reading it
        return perf_counter() - start
    
    return elapsed

In [44]:
x = timer()
x()

1.833400165196508e-05

In [45]:
y = timer()

In [46]:
print(x())
print(y())

27.070248125004582
14.175148333000834


In [48]:
print(a())
print(b())
print(x())
print(y())

276.1026563750056
234.20770774999983
108.12454312499904
95.2294179580058


In this example we are going to build an averager function that can average multiple values.

The twist is that we want to simply be able to feed numbers to that function and get a running average over time, not average a list which requires performing the same calculations (sum and count) over and over again.

In [1]:
class Averager:
    def __init__(self):
        self.numbers = []
    
    def add(self, number):
        self.numbers.append(number)
        total = sum(self.numbers)
        count = len(self.numbers)
        return total / count

In [2]:
a = Averager()

In [3]:
a.add(10)

10.0

In [4]:
a.add(20)

15.0

In [5]:
a.add(30)

20.0

We can do this using a closure as follows:

In [6]:
def averager():
    numbers = []
    def add(number):
        numbers.append(number)
        total = sum(numbers)
        count = len(numbers)
        return total / count
    return add

In [7]:
a = averager()

In [8]:
a(10)

10.0

In [9]:
a(20)

15.0

In [10]:
a(30)

20.0

Now, instead of storing a list and reclaculating `total` and `count` every time wer need the new average, we are going to store the running total and count and update each value each time a new value is added to the running average, and then return `total / count`.

Let's start with a class approach first, where we will use instance variables to store the running total and count and provide an instance method to add a new number and return the current average.

In [11]:
class Averager:
    def __init__(self):
        self._count = 0
        self._total = 0
    
    def add(self, value):
        self._total += value
        self._count += 1
        return self._total / self._count

In [12]:
a = Averager()

In [13]:
a.add(10)

10.0

In [14]:
a.add(20)

15.0

In [15]:
a.add(30)

20.0

Now, let's see how we might use a closure to achieve the same thing.

In [16]:
def averager():
    total = 0
    count = 0
    
    def add(value):
        nonlocal total, count
        total += value
        count += 1
        return 0 if count == 0 else total / count
    
    return add
        

In [17]:
a = averager()

In [18]:
a(10)

10.0

In [19]:
a(20)

15.0

In [20]:
a(30)

20.0

#### Generalizing this example

We saw that we were essentially able to convert a class to an equivalent functionality using closures. This is actually true in a much more general sense - very often, classes that define a single method (other than initializers) can be implemented using a closure instead.

Let's look at another example of this.

Suppose we want something that can keep track of the running elapsed time in seconds.

In [21]:
from time import perf_counter

In [22]:
class Timer:
    def __init__(self):
        self._start = perf_counter()
    
    def __call__(self):
        return (perf_counter() - self._start)

In [23]:
a = Timer()

Now wait a bit before running the next line of code:

In [24]:
a()

0.011695334544051804

Let's start another "timer":

In [25]:
b = Timer()

In [26]:
print(a())
print(b())

0.03528294403966765
0.011656054820407689


Now let's rewrite this using a closure instead:

In [27]:
def timer():
    start = perf_counter()
    
    def elapsed():
        # we don't even need to make start nonlocal 
        # since we are only reading it
        return perf_counter() - start
    
    return elapsed

In [28]:
x = timer()

In [29]:
x()

0.011068213438975016

In [30]:
y = timer()

In [31]:
print(x())
print(y())

0.03419096772236116
0.01164738619174141


In [32]:
print(a())
print(b())
print(x())
print(y())

0.10822159832175349
0.08475345336494494
0.0462381944113351
0.023573252079387305
